# Experiment 1: Baseline (No RAG)

In [1]:
# Setup
import sys
import json
from pathlib import Path
from typing import Dict, List, Any
from dataclasses import dataclass, asdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

sys.path.append('..')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Imports loaded")

Imports loaded


In [2]:
# Configuration
MODEL_PATH = Path("/home/sskaplun/study/genAI/kaggle/models/gemma-2-9b-it")
OUTPUT_DIR = Path("../evaluation/experiment_01")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# Generation parameters
TEMPERATURE = 0.7
MAX_NEW_TOKENS = 512

print(f"Model: {MODEL_PATH}")
print(f"Output: {OUTPUT_DIR}")
print(f"CUDA Available: {torch.cuda.is_available()}")

Model: /home/sskaplun/study/genAI/kaggle/models/gemma-2-9b-it
Output: ../evaluation/experiment_01
CUDA Available: True


In [3]:
@dataclass
class BaselineResponse:
    question: str
    answer: str
    temperature: float
    answer_length: int
    
    def to_dict(self):
        return asdict(self)

print("Dataclass defined")

Dataclass defined


## 1. Load Model

In [4]:
print("="*80)
print("LOADING GEMMA-2-9B-INSTRUCT")
print("="*80)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(str(MODEL_PATH))
print("Tokenizer loaded")

model = AutoModelForCausalLM.from_pretrained(
    str(MODEL_PATH),
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)
print("Model loaded")

if torch.cuda.is_available():
    memory_used = torch.cuda.memory_allocated() / 1024**3
    print(f"\nGPU Memory: {memory_used:.2f} GB")

LOADING GEMMA-2-9B-INSTRUCT


The tokenizer you are loading from '/home/sskaplun/study/genAI/kaggle/models/gemma-2-9b-it' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


`torch_dtype` is deprecated! Use `dtype` instead!


Tokenizer loaded


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded

GPU Memory: 5.71 GB


## 2. Define System Prompt & Generation

In [5]:
SYSTEM_PROMPT = """Ти — досвідчений викладач математики для українських учнів 10-11 класів.

Твоє завдання:
- Згенерувати математичну задачу з рішенням
- Використовувати ТІЛЬКИ українську мову
- Надати чітке пояснення та крок-за-кроком розв'язання
- Використовувати коректну українську математичну термінологію

Формат відповіді:
**Задача:** [текст задачі]

**Розв'язання:**
[покрокове рішення]

**Відповідь:** [фінальна відповідь]"""

print("System prompt defined")

System prompt defined


In [6]:
def generate_baseline(
    question: str,
    temperature: float = TEMPERATURE,
    max_new_tokens: int = MAX_NEW_TOKENS
) -> BaselineResponse:
    """Generate answer using LLM only (no RAG context)."""
    prompt = f"{SYSTEM_PROMPT}\n\nЗАПИТАННЯ:\n{question}\n\nТВОЯ ВІДПОВІДЬ:"
    
    messages = [{"role": "user", "content": prompt}]
    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9,
            do_sample=temperature > 0,
            pad_token_id=tokenizer.eos_token_id
        )
    
    answer = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    ).strip()
    
    return BaselineResponse(
        question=question,
        answer=answer,
        temperature=temperature,
        answer_length=len(answer)
    )

print("Generation function defined")

Generation function defined


## 3. Test Questions

In [7]:
from common import STANDARD_TEST_QUESTIONS, EVALUATION_DATASET

TEST_QUESTIONS = STANDARD_TEST_QUESTIONS
print(f"Test set: {len(TEST_QUESTIONS)} questions")

# Create mapping of questions to expected answers
question_to_expected = {q['input']: q['expected_answer'] for q in EVALUATION_DATASET}
print(f"Expected answers loaded for {len(question_to_expected)} questions")

Test set: 15 questions
Expected answers loaded for 10 questions


## 4. Run Baseline Experiment

In [8]:
print("="*80)
print("RUNNING BASELINE EXPERIMENT (NO RAG)")
print("="*80)

responses = []

for i, question in enumerate(TEST_QUESTIONS, 1):
    print(f"\n[{i}/{len(TEST_QUESTIONS)}] {question}")
    print("-"*80)
    
    response = generate_baseline(question)
    responses.append(response)
    
    print(f"\n{response.answer}")
    print(f"\nLength: {response.answer_length} chars")

print(f"\n{'='*80}")
print(f"Generated {len(responses)} responses")
print("="*80)

RUNNING BASELINE EXPERIMENT (NO RAG)

[1/15] Квадратні рівняння
--------------------------------------------------------------------------------



**Задача:**

На площі перед школою планують побудувати квітник у формі прямокутника.  Знайдіть сторони прямокутника, якщо його периметр дорівнює 36 метрів, а площа – 75 квадратних метрів.

**Розв'язання:**

1. **Позначимо сторони прямокутника:** 
   Нехай довжина прямокутника дорівнює *x* метрів, а ширина – *y* метрів.

2. **Запишемо рівняння за заданими умовами:**
   * **Периметр:** 2*x* + 2*y* = 36 
   * **Площа:** *x* * *y* = 75

3. **Розв'яжемо систему рівнянь:**
   * З першого рівняння можна виразити *x*: *x* = 18 - *y*
   * Підставимо цю вираз для *x* у друге рівняння: (18 - *y*) * *y* = 75
   * Розкриємо дужки: 18*y* - *y*² = 75
   * Перепишемо рівняння у стандартний вид: *y*² - 18*y* + 75 = 0
   * Розв'яжемо квадратне рівняння за формулою:
     *y* = (-b ± √(b² - 4ac)) / 2a, де a = 1, b = -18, c = 75
     *y* = (18 ± √((-18)² - 4 * 1 * 75)) / (2 * 1)
     *y* = (18 ± √(324 - 300)) / 2
     *y* = (18 ± √24) / 2
     *y* = (18 ± 2√6) / 2
     *y* = 9 ± √6
   * Оскільки ширина не


**Задача:** 

Марія хоче побудувати прямокутний садок з площею 120 м². Одну сторону садка вона хоче зробити на 5 метрів довшою за іншу. Знайдіть довжини сторін садка.

**Розв'язання:**

1. **Позначимо змінні:**
   * Нехай x - довжина коротшої сторони садка.
   * Тоді x + 5 - довжина довшої сторони садка.

2. **Утворимо рівняння:**
   * Площа прямокутника дорівнює добутку його сторін: x(x + 5) = 120.

3. **Розв'яжемо рівняння:**
   * Розкриємо дужки: x² + 5x = 120.
   * Перепишемо рівняння у стандартному вигляді: x² + 5x - 120 = 0.
   * Знайдемо коріння рівняння за допомогою формули:
     x = (-b ± √(b² - 4ac)) / 2a,
     де a = 1, b = 5, c = -120.
   * Підставимо значення: x = (-5 ± √(5² - 4 * 1 * -120)) / (2 * 1)
   * x = (-5 ± √(525)) / 2
   * x = (-5 ± 5√21) / 2

4. **Знайдемо довжини сторін:**
   * x1 = (-5 + 5√21) / 2 ≈ 9,6 м - довжина коротшої сторони
   * x2 = (-5 - 5√21) / 2 ≈ -14,6 м - відкидаємо, оскільки довжина не може бути від'ємною.

   * Довжина довшої сторони: x1 + 5 ≈


**Задача:**

На площі перед школою вирішили висадити квітники у вигляді квадратів. Один з квітників має площу 25 м², а сума довжин його сторін дорівнює 14 метрам. Знайдіть сторони цього квітника.

**Розв'язання:**

1. **Позначимо сторони квадрата:** Нехай сторона квадрата дорівнює *x* метрів.

2. **Сформулюємо рівняння:**
    * Площа квадрата дорівнює квадрату сторони:  *x²* = 25 м²
    * Сума довжин сторін квадрата дорівнює 4 *x*: 4*x* = 14 м

3. **Розв'яжемо рівняння:**
    * З першого рівняння знаходимо *x*:  *x* = √25 = 5 м

4. **Перевірка:**
    * Площа квадрата з стороною 5 м: 5 * 5 = 25 м²
    * Сума довжин сторін квадрата з стороною 5 м: 4 * 5 = 20 м

5. **Висновок:**

Сторони квітника дорівнюють **5 метрів**.



**Відповідь:** Сторони квітника дорівнюють 5 метрів.

Length: 783 chars

[4/15] Теорема Піфагора
--------------------------------------------------------------------------------



**Задача:**

У прямокутному трикутнику ABC, кут C – прямий, довжина гіпотенузи AB становить 13 см, а довжина катета AC дорівнює 5 см. Знайдіть довжину катета BC.

**Розв'язання:**

1. **Запишемо теорему Піфагора:** 
   В прямокутному трикутнику квадрат гіпотенузи дорівнює сумі квадратів катетів: AB² = AC² + BC²

2. **Підставимо відомі значення:**
   13² = 5² + BC²

3. **Обчислимо квадрати:**
   169 = 25 + BC²

4. **Виконемо перетворення:**
   BC² = 169 - 25

5. **Знайдемо квадратний корінь:**
   BC = √144

6. **Обчислимо:**
   BC = 12 см

**Відповідь:** Довжина катета BC дорівнює 12 см.

Length: 593 chars

[5/15] Теорема Піфагора
--------------------------------------------------------------------------------


...


**Задача:**  У ставку, що має форму кулі, плаває гусак. Діаметр ставка дорівнює 10 метрів. Який об'єм води в ставку?

**Розв'язання:**

1. **Визначимо формулу для обчислення об'єму кулі:**

   Об'єм кулі (V) = (4/3) * π * r³, де r - радіус кулі.

2. **Обчислимо радіус кулі:**

   Радіус (r) = діаметр / 2 = 10 м / 2 = 5 м

3. **Підставимо значення радіуса в формулу для обчислення об'єму:**

   V = (4/3) * π * (5 м)³ = (4/3) * π * 125 м³

4. **Обчислимо об'єм:**

   V ≈ 523.6 м³

**Відповідь:** Об'єм води в ставку дорівнює приблизно 523,6 кубічних метрів.

Length: 559 chars

[12/15] Об'єм кулі
--------------------------------------------------------------------------------



**Задача:** 

Куля має радіус 7 см. Знайдіть її об'єм.

**Розв'язання:**

1. **Нагадаємо формулу обчислення об'єму кулі:**
   V = (4/3) * π * r³, де:
   * V - об'єм кулі
   * π ≈ 3,14 - математична константа
   * r - радіус кулі

2. **Підставимо значення радіуса в формулу:**
   V = (4/3) * 3,14 * 7³

3. **Обчислимо куб радіуса:**
   V = (4/3) * 3,14 * 343

4. **Обчислимо добуток:**
   V ≈ 1436,03 см³

**Відповідь:** Об'єм кулі дорівнює приблизно 1436,03 см³.

Length: 462 chars

[13/15] Логарифми
--------------------------------------------------------------------------------



**Задача:**

На підприємстві виготовляють вироби з металу.  На початковому етапі виробництва кількість вироби в день зростає за формулою  $y = 2^x$, де *x* - кількість днів виробництва. Скільки днів знадобиться, щоб кількість вироблених виробів досягла 128?

**Розв'язання:**

1. **Визначимо, що нам потрібно знайти.** Нам потрібно знайти значення *x* (кількість днів), коли *y* (кількість вироблених виробів) дорівнює 128.

2. **Запишемо рівняння:** маємо рівняння $2^x = 128$.

3. **Знайдемо логарифм обох частин рівняння:** щоб ізолювати *x*, візьмемо логарифм з основа 2 від обох частин рівняння:  
   $\log_2(2^x) = \log_2(128)$

4. **Використовуємо властивість логарифмів:** згідно з властивістю логарифмів $\log_a(a^b) = b$, маємо: 
   $x = \log_2(128)$

5. **Обчислимо логарифм:** знаходимо значення $\log_2(128)$. Ми знаємо, що $2^7 = 128$, отже $\log_2(128) = 7$.

**Відповідь:**

Для того, щоб кількість вироблених виробів досягла 128, знадобиться **7 днів**.

Length: 970 chars

[14/15] 


**Задача:** 

На підприємстві випускають електронні книги.  Спочатку було продано 1000 примірників.  Через кожні 3 місяці кількість проданих електронних книг збільшувалася на 20%.  Через скільки місяців буде продано 5000 електронних книг?

**Розв'язання:**

1. **Зрозуміємо, що логарифми можуть нам допомогти розв'язати цю задачу.** Задача стосується експоненціального зростання, а логарифми є зворотньою операцією до експоненціального зростання.

2. **Визначимо формулу для експоненціального зростання.**  Нехай x - кількість місяців, після яких буде продано 5000 електронних книг. Тоді кількість проданих електронних книг за формулою буде:

  1000 * (1 + 0.20)^(x/3) = 5000

3. **Розв'яжемо рівняння.**
    * 1.02^(x/3) = 5
    * Щоб знайти x, застосуємо логарифм.  Використовуємо логарифм з основою 1.02:
    * log₁.₀₂(5) = x/3
    * x = 3 * log₁.₀₂(5)

4. **Обчислимо x.** 
    * Використовуючи калькулятор, знаходимо log₁.₀₂(5) ≈ 15.6.
    * x = 3 * 15.6 ≈ 46.8

5. **Округлимо x до найближчого


**Задача:** 

У фізиці відомо, що час, який потрібно для розпаду певного ізотопу, прямо пропорційний логарифму відношення початкової кількості ізотопу до кількості, що залишилась після розпаду. Якщо початкова кількість ізотопу становила 100 г, а через 10 років залишилося 50 г, знайдіть час, який потрібно для розпаду до 25 г.

**Розв'язання:**

1. **Визначимо змінні:**
   *  Нехай *t* - час розпаду (в роках).
   *  Нехай *N₀* - початкова кількість ізотопу (100 г).
   *  Нехай *N* - кількість ізотопу після розпаду.

2. **Запишемо формулу прямо пропорційності:**
   *  *t* ∝ log₂( *N₀* / *N* )

3. **Введемо коефіцієнт пропорційності (k):**
   *  *t* = *k* * log₂( *N₀* / *N* )

4. **Знайдемо *k* за відомими даними:**
   *  Через 10 років (*t* = 10 років) залишилося 50 г (*N* = 50 г). Підставимо ці значення у формулу:
      *  10 = *k* * log₂(100 / 50)
      *  10 = *k* * log₂(2)
      *  10 = *k* * 1
      *  *k* = 10

5. **Знайдемо час розпаду до 25 г:**
   *  *t* = *k* * log₂( *N₀* / *N*

## 5. Evaluation

In [9]:
import common

print("Evaluation functions loaded from common.py")

Evaluation functions loaded from common.py


In [10]:
# Evaluate all responses
print("="*80)
print("EVALUATION")
print("="*80)

evaluations = []

for i, response in enumerate(responses, 1):
    # Get expected answer for this question
    expected = question_to_expected.get(response.question, None)
    
    # Evaluate with expected answer for correctness
    metrics = common.evaluate_baseline(
        response.answer, 
        response.answer_length,
        expected_answer=expected
    )
    
    evaluations.append({
        'question': response.question,
        'metrics': metrics,
        'answer_length': response.answer_length,
        'expected_answer': expected
    })
    
    print(f"\n{i}. {response.question[:50]}...")
    print(f"   Overall: {metrics['overall_score']:.3f} | "
          f"Ukrainian: {metrics['ukrainian_ratio']:.3f} | "
          f"Correctness: {metrics['correctness']:.3f} | "
          f"Structure: {metrics['has_structure']}")

# Summary statistics
print(f"\n{'='*80}")
print("SUMMARY")
print("="*80)

avg_metrics = {
    'overall_score': np.mean([e['metrics']['overall_score'] for e in evaluations]),
    'ukrainian_ratio': np.mean([e['metrics']['ukrainian_ratio'] for e in evaluations]),
    'completeness': np.mean([e['metrics']['completeness'] for e in evaluations]),
    'structure_rate': sum(e['metrics']['has_structure'] for e in evaluations) / len(evaluations),
    'correctness': np.mean([e['metrics']['correctness'] for e in evaluations])
}

for key, value in avg_metrics.items():
    print(f"  {key:20s}: {value:.3f}")

EVALUATION

1. Квадратні рівняння...
   Overall: 0.586 | Ukrainian: 0.942 | Correctness: 1.000 | Structure: False

2. Квадратні рівняння...
   Overall: 0.690 | Ukrainian: 0.960 | Correctness: 1.000 | Structure: True

3. Квадратні рівняння...
   Overall: 0.697 | Ukrainian: 0.988 | Correctness: 1.000 | Structure: True

4. Теорема Піфагора...
   Overall: 0.531 | Ukrainian: 0.924 | Correctness: 0.000 | Structure: True

5. Теорема Піфагора...
   Overall: 0.534 | Ukrainian: 0.937 | Correctness: 0.000 | Structure: True

6. Теорема Піфагора...
   Overall: 0.535 | Ukrainian: 0.940 | Correctness: 0.000 | Structure: True

7. Площа трикутника...
   Overall: 0.525 | Ukrainian: 0.898 | Correctness: 0.000 | Structure: True

8. Площа трикутника...
   Overall: 0.541 | Ukrainian: 0.964 | Correctness: 0.000 | Structure: True

9. Площа трикутника...
   Overall: 0.535 | Ukrainian: 0.942 | Correctness: 0.000 | Structure: True

10. Об'єм кулі...
   Overall: 0.537 | Ukrainian: 0.948 | Correctness: 0.000 | Str

## 6. Visualization

In [11]:
# Create dataframe
df = pd.DataFrame([
    {
        'question_num': i+1,
        'question': e['question'][:40] + '...',
        'overall': e['metrics']['overall_score'],
        'ukrainian': e['metrics']['ukrainian_ratio'],
        'structure': int(e['metrics']['has_structure']),
        'length': e['answer_length']
    }
    for i, e in enumerate(evaluations)
])

print(df.to_string())

    question_num               question   overall  ukrainian  structure  length
0              1  Квадратні рівняння...  0.585586   0.942342          0    1089
1              2  Квадратні рівняння...  0.689948   0.959790          1    1083
2              3  Квадратні рівняння...  0.697024   0.988095          1     783
3              4    Теорема Піфагора...  0.530978   0.923913          1     593
4              5    Теорема Піфагора...  0.534205   0.936819          1     695
5              6    Теорема Піфагора...  0.534899   0.939597          1     689
6              7    Площа трикутника...  0.524586   0.898345          1     769
7              8    Площа трикутника...  0.540922   0.963687          1     575
8              9    Площа трикутника...  0.535445   0.941781          1     505
9             10          Об'єм кулі...  0.536967   0.947867          1     402
10            11          Об'єм кулі...  0.542902   0.971609          1     559
11            12          Об'єм кулі... 

In [12]:
common.create_metrics_visualization(
    evaluations=evaluations,
    avg_metrics=avg_metrics,
    output_path=OUTPUT_DIR / 'baseline_metrics.png',
    experiment_name='Baseline',
    metric_names=['ukrainian_ratio', 'completeness', 'correctness', 'structure_rate']
)

plt.show()

Visualization saved to ../evaluation/experiment_01/baseline_metrics.png


## 7. Save Results

In [13]:
results = {
    'experiment': 'baseline_no_rag',
    'description': 'LLM-only generation without retrieval context',
    'model': 'gemma-2-9b-it',
    'temperature': TEMPERATURE,
    'avg_metrics': avg_metrics,
    'responses': [r.to_dict() for r in responses],
    'evaluations': evaluations
}

with open(OUTPUT_DIR / 'results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

df.to_csv(OUTPUT_DIR / 'evaluation.csv', index=False)

print(f"Results saved to {OUTPUT_DIR}")
print("\n" + "="*80)
print("EXPERIMENT 1 COMPLETE")
print("="*80)
print(f"\nOverall Score: {avg_metrics['overall_score']:.3f}")
print(f"Ukrainian Ratio: {avg_metrics['ukrainian_ratio']:.3f}")
print(f"Correctness: {avg_metrics['correctness']:.3f}")
print(f"Structure Rate: {avg_metrics['structure_rate']:.3f}")

Results saved to ../evaluation/experiment_01

EXPERIMENT 1 COMPLETE

Overall Score: 0.560
Ukrainian Ratio: 0.948
Correctness: 0.200
Structure Rate: 0.933
